In [1]:
import random, shutil, os
import numpy as np
import pandas as pd

from mido import MidiFile
from glob import glob

In [4]:
def create_only_folders(dir, files):
    return [f for f in files if os.path.isfile(os.path.join(dir,f))]


shutil.copytree('/Users/cooky/Desktop/groove', './milisec_dataset', ignore=create_only_folders)

'./milisec_dataset'

In [5]:
midi_list = glob('/Users/cooky/Desktop/groove/**/**/*.mid')

In [43]:
def kit_classify(note):
    cat = ''
    if note in [36]: # kick
        cat = 0
    elif note in [38, 40, 37]: # snare 
        cat = 1
    elif note in [48, 50]: # High Tom
        # cat = 2
        cat = 2 # Replace HT -> MT
    elif note in [45, 47]: # Mid Tom
        # cat = 3
        cat = 2
    elif note in [43, 58]: # Low Tom
        # cat = 4
        cat = 3
    elif note in [46, 26]: # Open Hat
        # cat = 5
        cat = 4
    elif note in [42, 22, 44]: # Closed Hat
        # cat = 6
        cat = 5
    elif note in [49, 55, 57, 52]: # Crash
        # cat = 7 
        cat = 6 # Replace Cr -> Ri
    elif note in [51, 59, 53]: # Ride
        cat = 6
    else:
        print(note)

    return cat

In [46]:
# mid2 = MidiFile('/Users/cooky/Desktop/groove/drummer7/session1/3_rock_86_fill_4-4.mid', clip=True)
count = 0

for midi in midi_list:
    mid = MidiFile(midi, clip=True)
    on_sec = 0
    event = []

    for msg in mid.tracks[0]:
            note = str(msg).split()
            if len(note) == 5:
                on_sec += msg.dict()['time']

                if note[0] == 'note_on':
                    if [round(on_sec/10), msg.dict()['note']] not in event:
                        event.append([round(on_sec/10), msg.dict()['note']])
    count += 1
    # print(len(event))
    # print(event[-1])
    # print('---')
    # if count > 2:
    #     break
    
    beat = pd.DataFrame(0, columns=np.arange(7), index=np.arange(event[-1][0] + 1))

    # print(beat)
    # print('0')

    for e in event:
        beat.loc[e[0],kit_classify(e[1])] = 1

    # if beat.shape[1] < 32:
    #     for i in range(beat.shape[1], 32):
    #         beat[i] = 0

    # print(beat)
    # print('----')
    # if count > 1:
    #     break
    beat.to_csv('./milisec_dataset'+midi[27:]+'.csv.gzip', index = False, compression='gzip')

In [47]:
beat

,0,1,2,3,4,5,6
0,1,0,0,0,0,0,1
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
358,0,0,0,0,0,0,0
359,0,0,0,0,0,0,0
360,0,0,0,0,0,0,0
361,0,0,0,0,0,0,0


In [ ]:
csv_list = glob('./milisec_dataset/**/**/*.mid.csv.gzip')

In [ ]:
fill_cnt = 1
beat_cnt = 1

for csv in csv_list:
    imsi = pd.read_csv(csv, compression='gzip')
    if 'beat' in csv.lower():
        if imsi.shape[1] == 32:
            imsi.to_csv('./dataset/beat/'+str(beat_cnt)+'.csv.gzip', index=False, compression='gzip')
            imsi[random.choices(imsi.columns, k=3)] = 0
            imsi.to_csv('./masked_dataset/beat/'+str(beat_cnt)+'.csv.gzip', index=False, compression='gzip')
            beat_cnt += 1
        else:
            for i in range(imsi.shape[1] // 16 // 2):
                a = random.randrange(0, imsi.shape[1]-32)
                two_bars = imsi.iloc[:,a:a+32]
                two_bars.to_csv('./dataset/beat/'+str(beat_cnt)+'.csv.gzip', index=False, compression='gzip')
                two_bars[random.choices(two_bars.columns, k=3)] = 0
                two_bars.to_csv('./masked_dataset/beat/'+str(beat_cnt)+'.csv.gzip', index=False, compression='gzip')
                beat_cnt += 1
    elif 'fill' in csv.lower():
        if imsi.shape[1] == 32:
            imsi.to_csv('./dataset/fill/'+str(fill_cnt)+'.csv.gzip', index=False, compression='gzip')
            imsi[random.choices(imsi.columns, k=3)] = 0
            imsi.to_csv('./masked_dataset/fill/'+str(fill_cnt)+'.csv.gzip', index=False, compression='gzip')
            fill_cnt += 1
        else:
            for i in range(imsi.shape[1] // 16 // 2):
                a = random.randrange(0, imsi.shape[1]-32)
                two_bars = imsi.iloc[:,a:a+32]
                two_bars.to_csv('./dataset/fill/'+str(fill_cnt)+'.csv.gzip', index=False, compression='gzip')
                two_bars[random.choices(two_bars.columns, k=3)] = 0
                two_bars.to_csv('./masked_dataset/fill/'+str(fill_cnt)+'.csv.gzip', index=False, compression='gzip')
                fill_cnt += 1
    

        



/var/folders/wn/fhcj6_9d2qb5d0pw_xw20ky80000gn/T/ipykernel_1498/545134225.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  two_bars[random.choices(two_bars.columns, k=3)] = 0
/var/folders/wn/fhcj6_9d2qb5d0pw_xw20ky80000gn/T/ipykernel_1498/545134225.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  two_bars[random.choices(two_bars.columns, k=3)] = 0
/var/folders/wn/fhcj6_9d2qb5d0pw_xw20ky80000gn/T/ipykernel_1498/545134225.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice